In [87]:
import os
from typing import TypedDict, List, Dict, Annotated
import operator

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI


from langgraph.graph import StateGraph, END
from google import genai
from google.genai import types
    

gemini_api_key = os.getenv("GEMINI_API_KEY")


The goal of  our project was if we were able to build an agentic system that could produce books that can be uploaded to Amazon and if we could somewhat automate the process of writing book. I personally have private interest in literature so I really wonder if this is sensible and if it truly makes sense to automate this process. 

The idea of the workflow is simple.  
The user query is taken. 
Then an outline is generated. 
This outline is then used in a loop to generate the individual chapters.
Then the whole book is concatenated and the book automatically uploaded

In [88]:
# --- 1. Define the State ---
# This state is simplified to focus on the narrative structure: premise -> outline -> chapters.

class BookWriterState(TypedDict):
    """
    Represents the state of our book writing graph.

    Attributes:
        book_topic: The initial high-level topic for the book.
        book_premise: A more detailed premise or summary generated by the LLM.
        book_outline: A dictionary containing the title and a list of chapter titles.
        chapter_index: An index to keep track of the current chapter being written.
        draft_chapters: A list to store the written content of each chapter.
        book_title: The title of the book.
        book_description: A brief description of the book.
        author_name: The name of the author.
        end_writing: A flag to indicate if the writing process is complete.
        error: A field to store any errors that occur.
        book_cover: A field to store the book cover image.
    """
    book_topic: str
    book_premise: str
    book_outline: Dict
    chapter_index: int
    book_title: str
    book_description: str
    # We use Annotated and operator.add to accumulate chapters in the list.
    draft_chapters: Annotated[List[str], operator.add]
    error: str
    end_writing: bool
    author_name: str
    book_cover_file_path: str





As typical in LangGraph you need to define the individual nodes. We have a node for each task of the writing process. We quickly test the LLM here , just so we know it works

In [89]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash", temperature=0.2, max_output_tokens=65536,  api_key=gemini_api_key)

response = llm.invoke("Just say 'Hello, World!'")
print(response)
json_parser = JsonOutputParser()
str_parser = StrOutputParser()
BOOK_COVERS = "book_covers"
os.makedirs(BOOK_COVERS, exist_ok=True)
BOOKS = "books"
os.makedirs(BOOKS, exist_ok=True)

# You'll need to install the library: pip install google-cloud-aiplatform
import vertexai
from vertexai.vision_models import ImageGenerationModel

# --- Make sure to initialize the Vertex AI SDK ---
# This should be done once when your application starts.
# You'll need to be authenticated with Google Cloud.
# e.g., run `gcloud auth application-default login` in your terminal.
try:
    # Provide your project and location
    vertexai.init(project="romance-quest", location="europe-west4")
except Exception as e:
    print(f"Vertex AI initialization failed. Please check your configuration. Error: {e}")
# ----------------------------------------------------


def generate_premise_node(state: BookWriterState) -> Dict:
    """Generates a compelling book premise from a high-level topic."""
    print("--- Generating Book Premise ---")
    try:
        prompt = PromptTemplate(
            template="Generate a compelling one-paragraph book premise based on this topic: {topic}",
            input_variables=["topic"],
        )
        chain = prompt | llm | str_parser
        print("current book topic:", state["book_topic"])
        premise = chain.invoke({"topic": state["book_topic"]})
        print(f"Generated Premise: {premise}")
        prompt = PromptTemplate(
            template="Generate a nice title for a book based on this premise: {premise}. Just give me the title, nothing else.",
            input_variables=["premise"],
        )
        title_chain = prompt | llm | str_parser
        book_title = title_chain.invoke({"premise": premise})
        print(f"Generated Title: {book_title}")
        
        return {"book_premise": premise, "book_title": book_title} 
    except Exception as e:
        return {"error": f"Failed to generate premise: {e}"}
    
def generate_cover_node(state: BookWriterState) -> Dict:

    
    try:
        client = genai.Client( api_key=gemini_api_key)

        prompt = f"Generate a book cover image based on this title: {state['book_title']} and premise {state['book_premise']}. The authors name is {state['author_name']}. Make it compelling."

        response = client.models.generate_images(
        model='imagen-4.0-ultra-generate-preview-06-06',
        prompt=prompt,
        config=types.GenerateImagesConfig(
            number_of_images= 1,
        ))
    


        for generated_image in response.generated_images:
            book_cover_file_path = f"{state['book_title']}_cover.jpg"
            cover_path = os.path.join(BOOK_COVERS, book_cover_file_path)
            generated_image.image.save(cover_path)

            print(f"Saved book cover image to: {cover_path}")

        return {"book_cover_file_path": cover_path}
    except Exception as e:
        return {"error": f"Failed to generate book cover: {e}"}

def generate_outline_node(state: BookWriterState) -> Dict:
    """Generates a book outline based on the premise."""
    print("\n--- Generating Book Outline ---")
    try:
        print(f"now creating outline for premise: {state['book_premise']}")
        prompt = PromptTemplate(
            template="Based on this premise, create a book outline with a title and a list of chapter titles and a  description for each chapter that makes the narrative structure of this chapter clear. Each chapter should follow a 5-part dramatic structure. (JSON format) use 'title' and 'description'. Premise: {premise}.",
            input_variables=["premise"],
        )
        chain = prompt | llm  | str_parser | json_parser
        outline = chain.invoke({"premise": state["book_premise"]})
        print(f"Generated Outline: {outline}")
        print(f"Amount of Chapters: {len(outline.get('chapters', []))}")
       

        return {"book_outline": outline, }
    except Exception as e:
        return {"error": f"Failed to generate outline: {e}"}

def write_chapter_node(state: BookWriterState) -> Dict:
    """Writes the content for a single chapter based on the outline."""
    print("\n--- Writing Chapter ---")
    #print(f"Current State: {state}")
    try:
        outline = state["book_outline"]
        chapter_index = state["chapter_index"]
        chapter_title = outline["chapters"][chapter_index]["title"]
        chapter_description = outline["chapters"][chapter_index]["description"]

        prompt = PromptTemplate(
            template="Write the content for Chapter {chapter_num}: '{chapter_title}' with description: {description}. The overall book premise is: {premise}. Please give me the a json with 'title' and content. Never use markdown and don't use *. Write a full chapter so around 10 pages",
            input_variables=["chapter_num", "chapter_title", "premise", "description"],
        )
        chain = prompt | llm | str_parser | json_parser
        chapter_content = chain.invoke({
            "chapter_num": chapter_index + 1,
            "chapter_title": chapter_title,
            "premise": state["book_premise"],
            "description": chapter_description
        })
        print(f"Wrote Content for: {chapter_title}")
        return {"draft_chapters": [chapter_content], "chapter_index": chapter_index + 1}
    except Exception as e:
        return {"error": f"Failed to write chapter: {e}"}


def end_writing_node(state: BookWriterState) -> Dict:
    """Finalizes the writing process."""
    print("\n--- Finalizing Writing Process ---")
    try:
        if state["draft_chapters"]:
            print("Writing completed successfully.")
            return { "end_writing": True}
        else:
            return {"error": "No chapters were written.", "end_writing": True}
    except Exception as e:
        return {"error": f"Failed to finalize writing: {e}", "end_writing": True}
    



def write_description_node(state: BookWriterState) -> Dict:
    print("\n--- Writing Book Description ---")
    try:
        prompt = PromptTemplate(
            template="This is a book I wrote: {draft_chapters}. It has the title {title}. Please write a compelling description for it. You can use HTML tags to format the description.  Just give me the description, nothing else.",
            input_variables=["title", "draft_chapters"],  # Include draft_chapters if needed for context
        )
        
        formatted_prompt = prompt.format(title=state.get("book_title", "[Untitled Book]"), draft_chapters=state.get("draft_chapters", []))

      

        response = llm.invoke(formatted_prompt)
        

        description = str_parser.invoke(response)
      
        return {"book_description": description}
    except Exception as e:
        return {"error": f"Failed to write description: {e}"}





content='Hello, World!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--2a666502-b224-4cf2-a7ef-0c5ff19f7b10-0' usage_metadata={'input_tokens': 8, 'output_tokens': 20, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 16}}


In [90]:


def should_continue_writing(state: BookWriterState) -> str:
    """Checks if there are more chapters to write."""
    print("--- Checking for More Chapters ---")
    next_index = state["chapter_index"]
    
    if next_index < len(state["book_outline"]["chapters"]):
        print("Decision: Continue with next chapter.")
        # Update the index in the state to process the next chapter
        return "continue_writing"
    else:
        print("Decision: All chapters written. End process.")
        return "end_writing"



    


In [91]:
# --- 5. Assemble the Graph ---

workflow = StateGraph(BookWriterState)

# Add nodes
workflow.add_node("generate_premise", generate_premise_node)
workflow.add_node("generate_outline", generate_outline_node)
workflow.add_node("write_chapter", write_chapter_node)
workflow.add_node("end_writing", end_writing_node)
workflow.add_node("write_description", write_description_node)
workflow.add_node("generate_cover_node", generate_cover_node)
# Define the flow
workflow.set_entry_point("generate_premise")
workflow.add_edge("generate_premise", "generate_outline")
workflow.add_edge("generate_outline", "generate_cover_node")
workflow.add_edge("generate_cover_node", "write_chapter")


# Add the conditional edge for the chapter writing loop
workflow.add_conditional_edges(
    "write_chapter",
    should_continue_writing,# A checkpointer is needed for state updates in conditional edges, but we can manage manually for this simple case
    {
        "continue_writing": "write_chapter", # If 'continue', loop back
        "end_writing": "write_description"               # If 'done', finish
    }
)


workflow.add_edge("write_description", "end_writing")  # End the workflow after writing the description

# Compile the graph
app = workflow.compile(checkpointer=None) 



In [92]:
# --- 6. Run the Graph ---


def run_graph(initial_input):
  
    state = initial_input.copy()
    
    #while True:

    
    output = app.invoke(state)
    if output.get("end_writing"):
            print("Graph finished.")
            return output
      
        
    

print("Please describe the topic of the book I should write:")
#input_topic = input().strip()
input_topic = "I want to generate a dark romance book. The target audience is lonely women of all ages. The book should appeal to the reader’s fantasies and can be very cliché. The story is set in the late 1990s. The young, rather plain Nadja works in an office in New York. The book is told entirely from her perspective. Her overbearing boss, Temple, is both a threat and an attraction to her, because he is very handsome. She cannot leave the job because she needs the money to take care of her sick brother Josi.However, Temple has a darker secret: he has an alternate personality called Darksider. In the middle of the story, she meets a mysterious new co-worker named John. From the moment they meet, it’s clear that they want each other. She flees with him to a love nest in Aspen.The following chapters are written from the perspective of the now unhinged Darksider, who follows Nadja’s and John’s trail. He grows jealous of the spicy things he learns about them. He surprises them during lovemaking.Back to Nadja’s perspective:John reveals that he is an undercover cop and must shoot Darksider. However, he cannot make the death public because he already has other problems with the force—he’s a bad boy.So Nadja and he bury the body.The epilogue takes place in Nadja’s and John’s large house, three years later. They are married. The children—twins, a boy and a girl, with John’s brilliant blue eyes and Nadja’s dark hair—play outside with John while Nadja cooks. She thinks about how much she loves John, how she will keep their dark secret forever, and how it binds them together."
initial_input = {
    "book_topic": input_topic,
    "chapter_index": 0,
    "draft_chapters": [],
    "author_name": "Johannes Barthold",
}

print("--- Starting Book Writer Graph ---")
final_state = run_graph(initial_input)

print("\n--- Graph Finished ---")
print(f"\nTitle: {final_state.get('book_outline', {}).get('title')}")
print("\nPremise:")
print(final_state.get('book_premise'))
print("\nDraft Chapters:")
for i, chapter in enumerate(final_state.get('draft_chapters', [])):
    print(f"--- Chapter {i+1} ---\n{chapter}\n")
print("\nBook Description:")
print(final_state.get('book_description', 'No description generated.'))
print("\nAuthor Name:")
print(final_state.get('author_name', 'No author name provided.'))
print("\n--- Book Writing Process Completed ---")

Please describe the topic of the book I should write:
--- Starting Book Writer Graph ---
--- Generating Book Premise ---
current book topic: I want to generate a dark romance book. The target audience is lonely women of all ages. The book should appeal to the reader’s fantasies and can be very cliché. The story is set in the late 1990s. The young, rather plain Nadja works in an office in New York. The book is told entirely from her perspective. Her overbearing boss, Temple, is both a threat and an attraction to her, because he is very handsome. She cannot leave the job because she needs the money to take care of her sick brother Josi.However, Temple has a darker secret: he has an alternate personality called Darksider. In the middle of the story, she meets a mysterious new co-worker named John. From the moment they meet, it’s clear that they want each other. She flees with him to a love nest in Aspen.The following chapters are written from the perspective of the now unhinged Darksider,

Now we have generated our book. We want to format is nicely though so we need to create an EBUP to publish this book.

In [93]:
from ebooklib import epub



def create_epub(state: BookWriterState, filename: str = "book.epub"):
    """Creates an EPUB file from the book state."""
    book = epub.EpubBook()

    # Set metadata
    book.set_title(state.get("book_title", "Untitled Book"))
    book.set_language("en")
    book.add_author(state.get("author_name", "Unknown Author"))



    # Add cover image
    cover_file_path = state.get("book_cover_file_path", "cover.jpg")
    print(f"Using cover file: {cover_file_path}")
    try:
       book.set_cover(cover_file_path, open(cover_file_path, 'rb').read())

    
    except FileNotFoundError:
        print("⚠️ Cover image 'cover.jpg' not found. Skipping cover.")

    # Add chapters
    draft_chapter = state.get("draft_chapters", [])
    #print(draft_chapter)
    chapter_items = []
    for chapter in draft_chapter:
        
        chapter_title = str(chapter["title"])
        #print(chapter_title)
        chapter_content = str(chapter["content"])
        #print(chapter_content)
        c = epub.EpubHtml(title=chapter_title, file_name=f"{chapter_title}.xhtml", lang="en")
        c.content = f"<h1>{chapter_title}</h1><p>{chapter_content}</p>"
        book.add_item(c)
        chapter_items.append(c)
         

    # Add navigation
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())
    book.toc = (
    (epub.Section('Chapters'), tuple(chapter_items)),
)

    book.spine = ['cover', 'nav'] + chapter_items

    
    

    
 
    # Save the EPUB file
    epub.write_epub(filename, book)
    print(f"✅ EPUB file '{filename}' created successfully.")

create_epub(final_state, os.path.join(BOOKS, f"{final_state['book_title']}.epub"))

Using cover file: book_covers/The Aspen Secret_cover.jpg
✅ EPUB file 'books/The Aspen Secret.epub' created successfully.
